# Infos - Links

### Youtube video explaining how to convert images and masks to tensor flow:

https://www.youtube.com/watch?v=C5CbsTDwQM0

Github link of the video: https://github.com/nikhilroxtomar/tf.data-TensorFlow-Input-Data-Pipeline-for-Semantic-Segmentation



### Links for notebooks and blogs used to prepare this notebook:

https://huggingface.co/docs/transformers/model_doc/segformer#

https://github.com/huggingface/notebooks/blob/main/examples/semantic_segmentation-tf.ipynb

https://huggingface.co/blog/fine-tune-segformer (I am not 100% sure if this blog is using tensor flow)

https://github.com/deep-diver/segformer-tf-transformers/blob/main/notebooks/TFSegFormer_Finetune.ipynb

I didn't use this one yet: https://github.com/deep-diver/segformer-tf-transformers/blob/main/notebooks/TFSegFormer_ONNX.ipynb

https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/image_classification.ipynb#scrollTo=Mp9xJcHP2TTP

Setting up a model from scratch on tensor flow: https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb#scrollTo=-47O6_GLdRuT



### Additional Info:
https://medium.com/geekculture/semantic-segmentation-with-segformer-2501543d2be4





# Load Transformers

In [2]:
# load transformers from HuggingFace to load the model. Later in the
## notebook, the model was downloaded via tenser flow. I am not sure 
### if we still need this cell. 

# Comment it out after you run it
#%pip install git+https://github.com/huggingface/transformers

In [3]:
# Ensure we have access to a GPU. Later for the cloud
# !nvidia-smi

In [4]:
# Install necessary packages. This will be moved to the requirements.txt later
# !pip install datasets==2.3.2
# !pip install numpy==1.23.0
# !pip install pillow==9.1.1
# !pip install matplotlib==3.5.2
#!pip install tensorflow==2.9.0 # inside requirement_dev we installed 2.13.0

In [5]:
# this cel was from a different notebook. 
## I need to check if we still need to install this. 

# comment out after running
# !pip install ipywidgets

In [6]:
import json
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# we do not need those because we have our own data
#from datasets import load_dataset, load_metric
#from huggingface_hub import cached_download, hf_hub_url

from tensorflow.keras.optimizers import Adam

from transformers import (
    DefaultDataCollator,
    SegformerFeatureExtractor,
    TFSegformerForSemanticSegmentation,
    create_optimizer,
)
from transformers.keras_callbacks import KerasMetricCallback

In [17]:
'''
For the purpose of this example, we will keep the runtime short
(2 epochs only). But we will provide a link to a model checkpoint
that was trained for 50 epochs with the exact same code.
'''
epochs = 2
lr = 0.00006
batch_size = 2

In [8]:
# skip this cell for now


# comment out after running
#!pip install tensorflow-datasets

# Preparing images for Tensor Flow

## Preparing Imges: Option1

You can run 'python data_copy.py' on Terminal or do as in the next 2 cells and run the script on notebook

In [9]:
#skip this cell for now 

# change the path to your 'data_copy.py'
# script_path = '/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/python_files/data_copy.py'

In [10]:
# I used Option 2
# change the path to your 'data_copy.py'

# !python /Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/python_files/data_copy.py

Images: 12 - Masks: 12
/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/data/xBD_test_subset_reorganized/images/mexico-earthquake_00000005_post_disaster.png /Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/data/xBD_test_subset_reorganized/masks/mexico-earthquake_00000005_pre_disaster.png
/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/data/xBD_test_subset_reorganized/images/santa-rosa-wildfire_00000028_post_disaster.png /Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/data/xBD_test_subset_reorganized/masks/santa-rosa-wildfire_00000028_pre_disaster.png
/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/data/xBD_test_subset_reorganized/images/mexico-earthquake_00000022_post_disaster.png /Users/sevincjakab/neu

## Preparing Images: Option 2

Skip this step if you did option 1

In [11]:
# Data Loading and Preprocessing with TensorFlow

#This notebook demonstrates how to load and preprocess image and mask data using TensorFlow's tf.data API.

## Step 1: Load the Dataset - Image and Mask Paths

#Let's start by loading the dataset, which contains images and
# their corresponding masks. The paths to the images and masks will be stored in separate lists.

import numpy as np
import cv2
from glob import glob
import tensorflow as tf

# before running this cell, you need to reorganize your data folder.
## One folder should have 'images' and the other 'masks with labels'. 
### You can mix all the disasters. 


def load_data(path):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))

    return images, masks

# Define the path to the dataset directory
path = "/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/data/xBD_test_subset_reorganized"
images, masks = load_data(path)

print(f"Images: {len(images)} - Masks: {len(masks)}")


Images: 12 - Masks: 12


In [12]:
# functions to process data

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_mask(y)

        return x, y

    images, masks = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([256, 256, 3])
    masks.set_shape([256, 256, 3])

    return images, masks


In [13]:
def tf_dataset(x, y, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

# Create the TensorFlow dataset
dataset= tf_dataset(images, masks, batch=4)
ds=dataset
# Let's print the dimensions of the first batch of preprocessed images and masks
for x, y in ds:
    print(x.shape, y.shape)
    break

(4, 256, 256, 3) (4, 256, 256, 3)


In [14]:
# check values inside the tensors
x = x[0] # images
y = y[0] # masks with labels
print(x,y)

tf.Tensor(
[[[0.42352942 0.5411765  0.57254905]
  [0.4745098  0.5882353  0.6156863 ]
  [0.4392157  0.54509807 0.5764706 ]
  ...
  [0.27058825 0.32941177 0.28627452]
  [0.38431373 0.4627451  0.42352942]
  [0.21568628 0.28235295 0.2509804 ]]

 [[0.3372549  0.4392157  0.45882353]
  [0.43529412 0.54509807 0.57254905]
  [0.46666667 0.57254905 0.60784316]
  ...
  [0.25882354 0.29803923 0.2627451 ]
  [0.18039216 0.23921569 0.21176471]
  [0.1882353  0.24313726 0.21960784]]

 [[0.39607844 0.5019608  0.53333336]
  [0.38039216 0.49411765 0.52156866]
  [0.41960785 0.5372549  0.5686275 ]
  ...
  [0.18431373 0.24705882 0.2       ]
  [0.27058825 0.33333334 0.2784314 ]
  [0.21568628 0.27450982 0.24313726]]

 ...

 [[0.25882354 0.30588236 0.2627451 ]
  [0.34117648 0.41568628 0.39215687]
  [0.33333334 0.40392157 0.40392157]
  ...
  [0.43137255 0.4862745  0.43529412]
  [0.44313726 0.4862745  0.44313726]
  [0.42352942 0.4745098  0.43137255]]

 [[0.30980393 0.36862746 0.3647059 ]
  [0.28627452 0.36078432 0

# Train-Test Split

In [15]:
# FOR BIG DATASET
# ds = dataset.shuffle(seed=1, buffer_size=1000)

# # Calculate the number of samples for validation set
# val_size = int(len(images) * 0.5)  # 20% of the data for validation

# # Split the dataset into training and validation sets
# train_ds = ds.take(len(images) - val_size)
# val_ds = ds.skip(len(images) - val_size)

# # Optionally, you can further batch the validation dataset
# val_ds = val_ds.batch(batch_size)

# # Check the number of samples in each dataset
# print("Train samples:", len(list(train_ds)))
# print("Validation samples:", len(list(val_ds)))


In [19]:
# FOR SMALL DATASET
# Previously defined code for creating the dataset

ds = dataset.shuffle(seed=1, buffer_size=1000)

# Split the dataset into training and validation sets
train_ds = ds.take(2)
val_ds = ds.skip(2)

# Optionally, you can further batch the validation dataset
val_ds = val_ds.batch(batch_size)

# Check the number of samples in each dataset
print("Train samples:", len(list(train_ds)))
print("Validation samples:", len(list(val_ds)))


Train samples: 2
Validation samples: 1


In [20]:
# Get the first batch from the validation dataset
val_batch = next(iter(val_ds))

# Access the pixel_values from the validation batch
pixel_values = val_batch[0]
print(pixel_values)

tf.Tensor(
[[[[[0.         0.         0.        ]
    [0.         0.         0.        ]
    [0.         0.         0.        ]
    ...
    [0.29411766 0.3882353  0.2784314 ]
    [0.32156864 0.3647059  0.2784314 ]
    [0.21176471 0.23529412 0.1882353 ]]

   [[0.38039216 0.4627451  0.45882353]
    [0.44705883 0.52156866 0.52156866]
    [0.54509807 0.6313726  0.6509804 ]
    ...
    [0.23529412 0.35686275 0.24313726]
    [0.2784314  0.36078432 0.2627451 ]
    [0.29803923 0.31764707 0.24705882]]

   [[0.4392157  0.5137255  0.5176471 ]
    [0.4862745  0.54901963 0.56078434]
    [0.5019608  0.58431375 0.6039216 ]
    ...
    [0.27058825 0.38431373 0.26666668]
    [0.25882354 0.36078432 0.25490198]
    [0.21960784 0.2784314  0.20784314]]

   ...

   [[0.4862745  0.5686275  0.54509807]
    [0.42745098 0.5372549  0.49803922]
    [0.46666667 0.5803922  0.5372549 ]
    ...
    [0.21176471 0.29411766 0.23137255]
    [0.3137255  0.39607844 0.3137255 ]
    [0.2784314  0.32941177 0.2784314 ]]

   [[

In [21]:
# define labels

id2label = {
"no-damage": 0,
"minor-damage": 1,
"major-damage": 2,
"destroyed": 3
}

label2id = {
"0": "no-damage",
"1": "minor-damage",
"2": "major-damage",
"3": "destroyed"
}

num_labels = len(id2label)
num_labels

4

In [24]:
# install libraries to be able to install the evaluation metrics in the next cell
## Error!
!pip install datasets
from datasets import load_metric

ValueError: mutable default <class 'datasets.utils.version.Version'> for field version is not allowed: use default_factory

In [ ]:
# Error!

'''
Once the metric is loaded, you can use it to evaluate the performance 
of your model on a specific task, such as semantic segmentation, 
where the "mean_iou" (mean Intersection over Union) is a common evaluation metric.
'''

metric = load_metric("mean_iou")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # logits are of shape (batch_size, num_labels, height, width), so
    # we first transpose them to (batch_size, height, width, num_labels)
    logits = tf.transpose(logits, perm=[0, 2, 3, 1])
    # scale the logits to the size of the label
    logits_resized = tf.image.resize(
        logits,
        size=tf.shape(labels)[1:],
        method="bilinear",
    )
    # compute the prediction labels and compute the metric
    pred_labels = tf.argmax(logits_resized, axis=-1)
    metrics = metric.compute(
        predictions=pred_labels,
        references=labels,
        num_labels=num_labels,
        ignore_index=-1,
        reduce_labels=feature_extractor.reduce_labels,
    )
    return {"val_" + k: v for k, v in metrics.items()}


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics,
    eval_dataset=val_set,
    batch_size=batch_size,
    label_cols=["labels"],
)

NameError: name 'load_metric' is not defined

# Load the Model

In [ ]:
# load the model

model_checkpoint = "nvidia/mit-b0"
model = TFSegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,  # Will ensure the segmentation specific components are reinitialized.
)

optimizer = Adam(learning_rate=lr)
model.compile(optimizer)

2023-08-01 01:14:03.910591: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x147361b00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-08-01 01:14:03.910718: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-08-01 01:14:04.038005: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

model.summary()

Model: "tf_segformer_for_semantic_segmentation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 segformer (TFSegformerMain  multiple                  3319392   
 Layer)                                                          
                                                                 
 decode_head (TFSegformerDe  multiple                  396292    
 codeHead)                                                       
                                                                 
Total params: 3715684 (14.17 MB)
Trainable params: 3715172 (14.17 MB)
Non-trainable params: 512 (2.00 KB)
_________________________________________________________________


In [ ]:
train_ds.element_spec

(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))

# Train the Model

In [ ]:
# error!
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
)

Epoch 1/2


ValueError: in user code:

    File "/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/sevincjakab/neuefische_bootcamp/20230717-NewRepo-Capstone-Building_Damage/Capstone_Building_Damage/.venv/lib/python3.11/site-packages/transformers/modeling_tf_ut